### Importing Libraries

In [1]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
# import math
# 
# 
# 
# 
# import tensorflow.keras
# from tensorflow import keras
# from IPython.display import clear_output
# from IPython.utils import io

Directory which will contain the Training Images according to alphabet. Sub-directories of alphabets are already present 

*Change it to a directory of your convinience*

In [2]:
TRAIN_DIR = "C:/Jupyter/IBM/Dataset/training_set/"

Defining the classes of images that we wish to classify. *As the letters J and Z require motion we are excluding them here*

In [3]:
CATEGORIES = ['A','B','C','D','E','F','G','H','I']

### Preparing Training Dataset

Creating empty directory to store training data

In [4]:
trainImgs = []

Recursive function that reads in image using OpenCv and Resizes it and stores the image array in a list

In [5]:
for cat in CATEGORIES:
    path=os.path.join(TRAIN_DIR,cat) # Specifying the directory to take images from for each alphabet
    class_num = CATEGORIES.index(cat) # Using the index of the CATEGORIES list to assign a label to each image
    for img in os.listdir(path): # FOR loop which appends read images to the Training List
        img=cv2.imread(TRAIN_DIR+cat+'/'+img)
        resizedimg = cv2.resize(img, (224,224), interpolation= cv2.INTER_CUBIC)
        trainImgs.append([resizedimg,class_num])

Shuffling the Train Data so that the Neural Network doesn't develop a pattern recognition system

In [6]:
random.shuffle(trainImgs)

Seperating the Images and Labels

In [7]:
X = []
y = []
for item in trainImgs:
    X.append(item[0])
    y.append(item[1])
X=np.array(X)
y=np.array(y)

Normalizing the Images

In [8]:
Xnor = X/255.0

Train Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(Xnor, y, test_size=0.3)

### Model Construction

Specifying URL where MobileNetV2 resides

In [10]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

Specifying the Input Layer

In [11]:
mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))

Making sure that the Inner Layers remain same as we don't want to change any pre-training in the CNN.

In [12]:
mobile_net_layers.trainable = False

Constructing the model

In [13]:
model = tf.keras.Sequential([
  mobile_net_layers,
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(9,activation='softmax')
])

Compiling the model

In [14]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Model Training

model.fit(X_train, y_train, validation_data=(X_test, y_test))      

As we can see the model has good accuracy. So I will use the entire data to train the model this time instead of just the train data

In [15]:
model = tf.keras.Sequential([
  mobile_net_layers,
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(9,activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(Xnor, y, batch_size=5)      

3150/3150 [==============================] - 237s 74ms/step - loss: 0.0779 - accuracy: 0.9827


Saving the model

In [16]:
model.save('IBMASL.h5')

Loading the saved model *(This step was done for ease of use so that I didn't need to train the model everytime I wanted to make some changes)*

In [18]:
model=tf.keras.models.load_model('IBMASL.h5',custom_objects={'KerasLayer': hub.KerasLayer})

Creating Test Data

In [21]:
TEST_DIR = "C:/Jupyter/IBM/Dataset/test_set/"

In [22]:
testImgs = []
for cat in CATEGORIES:
    path=os.path.join(TEST_DIR,cat) # Specifying the directory to take images from for each alphabet
    class_num = CATEGORIES.index(cat) # Using the index of the CATEGORIES list to assign a label to each image
    for img in os.listdir(path): # FOR loop which appends read images to the Training List
        img=cv2.imread(TEST_DIR+cat+'/'+img)
        resizedimg = cv2.resize(img, (224,224), interpolation= cv2.INTER_CUBIC)
        testImgs.append([resizedimg,class_num])

In [23]:
X_test = []
y_test = []
for item in testImgs:
    X_test.append(item[0])
    y_test.append(item[1])
X_test=np.array(X_test)
y_test=np.array(y_test)

In [25]:
model.evaluate(X_test, y_test)

71/71 [==============================] - 23s 327ms/step - loss: 1.1766 - accuracy: 0.6187


[1.1765762567520142, 0.6186666488647461]